In [1]:
import importlib
import sys
sys.path.insert(0, '/cndd/fangming/CEMBA/snmcseq_dev')

from __init__ import *
from __init__jupyterlab import *
from scipy import sparse
import collections
import shutil

import snmcseq_utils
importlib.reload(snmcseq_utils)
import CEMBA_preproc_utils
importlib.reload(CEMBA_preproc_utils)



<module 'CEMBA_preproc_utils' from '/cndd/fangming/CEMBA/snmcseq_dev/CEMBA_preproc_utils.py'>

# prep
- metadata
- hvfeatures (recalculated)
- ```__init__datasets.py``` 
- config.py (somewhere else) 

In [2]:
log = snmcseq_utils.create_logger()
logging.info('*')

09/23/2019 06:05:13 PM *


In [4]:
config_dir = '/cndd/fangming/CEMBA/data/MOp_all/data_freeze_neurons_v2/' 
config_f = 'config_scf_8mods_data_freeze_2kb_190923.py' 
name = 'mop_8mods_0923_2kb'
DST_ROOT_DIR = '/cndd/fangming/CEMBA/data/MOp_all/data_freeze_neurons_subtypes_8mods_2kb'
first_round_cluster_col = 'cluster_joint_r0.1'

# import config and import datasets
sys.path.insert(0, config_dir)
exec("from {} import *".format(config_f.split('.')[0]))
from __init__datasets import *

meta_f = '{0}/{1}_metadata.tsv'
raw_f = '{0}/{1}_{2}raw.{3}'
hvftr_f = '{0}/{1}_hvfeatures.{2}'

In [5]:
metas = collections.OrderedDict()
for mod in mods_selected:
    metas[mod] = pd.read_csv(meta_f.format(DATA_DIR, mod), sep="\t").reset_index().set_index(settings[mod].cell_col)
    print(mod, metas[mod].shape)

snmcseq_gene (9366, 32)
snatac_gene (54844, 13)


/cndd/fangming/venvs/conda_dobro/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2728: DtypeWarning: Columns (64) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


smarter_cells (6244, 129)
smarter_nuclei (5911, 146)
10x_cells_v2 (121440, 8)
10x_cells_v3 (69727, 8)
10x_nuclei_v3 (39706, 8)
10x_nuclei_v3_macosko (101647, 19)


In [ ]:
gxc_raws = collections.OrderedDict()
for mod in mods_selected:
    logging.info("Read data {}...".format(mod))
    if settings[mod].mod_category == 'mc':
        f_gene = raw_f.format(DATA_DIR, mod, '', 'gene')
        f_cell = raw_f.format(DATA_DIR, mod, '', 'cell')
        f_data_c = raw_f.format(DATA_DIR, mod, 'CH_', 'npz')
        f_data_mc = raw_f.format(DATA_DIR, mod, 'mCH_', 'npz')
        gxc_raws[mod] = snmcseq_utils.load_gc_matrix_methylation(f_gene, f_cell, f_data_mc, f_data_c)
        
    else:
        f_gene = raw_f.format(DATA_DIR, mod, '', 'gene')
        f_cell = raw_f.format(DATA_DIR, mod, '', 'cell')
        f_data = raw_f.format(DATA_DIR, mod, '', 'npz')
        gxc_raws[mod] = snmcseq_utils.load_gc_matrix(f_gene, f_cell, f_data)
        

09/23/2019 06:06:49 PM Read data snmcseq_gene...
09/23/2019 06:07:32 PM Read data snatac_gene...
09/23/2019 06:07:50 PM Read data smarter_cells...
09/23/2019 06:07:57 PM Read data smarter_nuclei...
09/23/2019 06:08:00 PM Read data 10x_cells_v2...
09/23/2019 06:08:48 PM Read data 10x_cells_v3...


In [ ]:
f = output_clst_and_umap
df_info = pd.read_csv(f, sep="\t", index_col='sample')[[first_round_cluster_col, 'modality']] 
print(df_info.shape)
df_info.head()

In [ ]:
normalization_options = {
    'smarter_nuclei': 'TPM',
    'smarter_cells': 'TPM',
    'snatac_gene': 'TPM',
    'snmcseq_gene': 'MC',
    '10x_nuclei_v3_macosko': 'CPM',
    '10x_cells_v3': 'CPM',
    '10x_nuclei_v3': 'CPM',
    '10x_cells_v2': 'CPM',
}

# # gene id (abbr) as index
gene_annot_file = PATH_GENEBODY_ANNOTATION
gene_annot = pd.read_csv(gene_annot_file, sep="\t")
gene_annot['gene_id_abbr'] = gene_annot['gene_id'].apply(lambda x: x.split('.')[0])
gene_annot = gene_annot.set_index('gene_id_abbr')

gene_lengths_base = (gene_annot['end'] - gene_annot['start'])
print(gene_lengths_base.head())

In [ ]:
# dataset and config
if not os.path.isdir(DST_ROOT_DIR):
    os.makedirs(DST_ROOT_DIR)

logging.info("Prep data...")
for (mod, clst), df_sub in df_info.groupby(['modality', first_round_cluster_col]):
    if mod in mods_selected:
        print(mod, clst)
        ti = time.time()
        normalization_option = normalization_options[mod]

        _cells = df_sub.index.values
        dst_dir = os.path.join(DST_ROOT_DIR, "sub"+str(clst)) # directory name
        if not os.path.isdir(dst_dir):
            os.makedirs(dst_dir)
            src = os.path.join(DATA_DIR, '__init__datasets.py') 
            dst = os.path.join(dst_dir, '__init__datasets.py') 
            shutil.copyfile(src, dst)

        # meta and save meta
        meta_f_dst = meta_f.format(dst_dir, mod)
        meta = metas[mod].loc[_cells]
        meta.to_csv(meta_f_dst, sep="\t", header=True, index=True)

        if settings[mod].mod_category == 'mc':
            # split raw data
            _cell_idx = snmcseq_utils.get_index_from_array(gxc_raws[mod].cell, _cells)
            gxc_raw = GC_matrix(
                gxc_raws[mod].gene,
                gxc_raws[mod].cell[_cell_idx],
                {'mc': gxc_raws[mod].data['mc'].tocsc()[:, _cell_idx],
                 'c': gxc_raws[mod].data['c'].tocsc()[:, _cell_idx],
                },
            )
            # check meta cells agree with gxc cells
            assert np.all(meta.index.values == gxc_raw.cell)
            # check genes are uniq 
            assert len(gxc_raw.gene) == len(np.unique(gxc_raw.gene)) 
            
            # get hvftrs
            gxc_hvftr = CEMBA_preproc_utils.preproc_methylation(gxc_raw, meta,
                                                                global_value_col=settings[mod].global_mean, 
                                                                base_call_cutoff=20, 
                                                                sufficient_coverage_fraction=0.95,
                                                                hv_percentile=30,
                                                                n_qcut=10,
                                                               )

            # save data
            print(mod, "Saving to files {}".format(time.time()-ti))
            f_data = hvftr_f.format(dst_dir, mod, 'tsv') 
            gxc_hvftr.to_csv(f_data, sep="\t", index=True, header=True)
            logging.info("{} {} Total time used: {}".format(mod, clst, time.time()-ti))
            
        else:
            # split raw data
            _cell_idx = snmcseq_utils.get_index_from_array(gxc_raws[mod].cell, _cells)
            gxc_raw = GC_matrix(
                gxc_raws[mod].gene,
                gxc_raws[mod].cell[_cell_idx],
                gxc_raws[mod].data.tocsc()[:, _cell_idx],
            )
            # check meta cells agree with gxc cells
            assert np.all(meta.index.values == gxc_raw.cell)
            # check genes are uniq 
            assert len(gxc_raw.gene) == len(np.unique(gxc_raw.gene)) 

            # get hvftrs
            print(mod, "Preproc and get highly variable genes {}".format(time.time()-ti))
            if normalization_option == 'CPM':
                gxc_hvftr = CEMBA_preproc_utils.preproc_rna_cpm_based(
                                                 gxc_raw, 
                                                 sufficient_cell_coverage=0.01, 
                                                 hv_percentile=30, hv_ncut=10)
            elif normalization_option == 'TPM':
                gene_lengths = gene_lengths_base.reindex(gxc_raw.gene)
                gxc_hvftr = CEMBA_preproc_utils.preproc_rna_tpm_based(
                                                 gxc_raw, gene_lengths, impute_gene_lengths=True, 
                                                 sufficient_cell_coverage=0.01, 
                                                 hv_percentile=30, hv_ncut=10)
                
            # save data
            print(mod, "Saving to files {}".format(time.time()-ti))
            f_gene = hvftr_f.format(dst_dir, mod, 'gene') 
            f_cell = hvftr_f.format(dst_dir, mod, 'cell') 
            f_data = hvftr_f.format(dst_dir, mod, 'npz') 
            snmcseq_utils.save_gc_matrix(gxc_hvftr, f_gene, f_cell, f_data)
            logging.info("{} {} Total time used: {}".format(mod, clst, time.time()-ti))

    
    